In [1]:
import json
from pathlib import Path
from prettyprinter import cpprint
from langchain.document_loaders import JSONLoader

In [2]:
# 加载 json 文档，查看数据样式
books_computer = json.loads(Path('../resources/计算机科学.json').read_text())

print(len(books_computer))
cpprint(books_computer[:2])

10
[
    {
        'book_name': 'Python 编程入门',
        'author': 'Eric Matthes',
        'classify': '编程语言',
        'price': 59.99,
        'publish_date': '2020-03',
        'press': 'No Starch Press',
        'summary': '这本书是初学者学习 Python 编程的绝佳选择，包含大量示例和练习，逐步引导读者掌握 Python 编程基础。',
        'document_content':
            '    书名：《Python 编程入门》\n    作者：Eric Matthes\n    分类：编程语言\n    价格：59.99\n'
            '    发布年份：2020-03\n    出版社：No Starch Press\n    简介：这本书是初学者学习 Python '
            '编程的绝佳选择，包含大量示例和练习，逐步引导读者掌握 Python 编程基础。'
    },
    {
        'book_name': '深度学习',
        'author': 'Ian Goodfellow, Yoshua Bengio, Aaron Courville',
        'classify': '人工智能',
        'price': 89.99,
        'publish_date': '2016-11',
        'press': 'MIT Press',
        'summary': '这是一本关于深度学习的经典教材，覆盖深度学习的基本原理和应用。',
        'document_content':
            '    书名：《深度学习》\n    作者：Ian Goodfellow, Yoshua Bengio, Aaron Courville\n'
            '    分类：人工智能\n    价格：89.99\n    发布年份：2016-11\n    出版社：MIT Pres

In [3]:
# 数据处理，合并图书各项信息作为 document content
for book in books_computer:
    book['document_content'] = f"""\
    书名：《{book["book_name"]}》
    作者：{book["author"]}
    分类：{book["classify"]}
    价格：{book["price"]}
    发布年份：{book["publish_date"].split('/')[-1]}
    出版社：{book["press"]}
    简介：{book["summary"]}"""

with open('../resources/计算机科学.json', 'w') as f:
    f.write(json.dumps(books_computer))

In [4]:
# 文档 metadata 函数，用于关联对应的 Document
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata.update(record)
    if 'document_content' in metadata:
        del metadata['document_content']
    if 'source' in metadata:
        metadata['source'] = metadata['source'].split('/')[-1]
    return metadata

In [5]:
# 使用 JSONLoader 加载json文档
loader = JSONLoader('../resources/计算机科学.json', 
                    jq_schema='.[]',
                   content_key='document_content',
                   metadata_func=metadata_func)
documents = loader.load()
print(len(documents))
print(documents[0].page_content)

10
    书名：《Python 编程入门》
    作者：Eric Matthes
    分类：编程语言
    价格：59.99
    发布年份：2020-03
    出版社：No Starch Press
    简介：这本书是初学者学习 Python 编程的绝佳选择，包含大量示例和练习，逐步引导读者掌握 Python 编程基础。
